Script: to concat files and assign top EF
--

In [1]:
import pandas as pd
import os
import seaborn as sns

Function to filter our background exons
--

In [66]:
def filter_and_save_exons(type_ss):
    all_EFs_df=pd.DataFrame()
    directory_path = '/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/00_data/emp_bayes_output/'+type_ss+'_outputs/assigned_EFs/'
    files=os.listdir(directory_path)

    # Loop through each file in the directory
    for file_name in files:
        # Check if the entry is a file (not a directory)
        if file_name!= '.ipynb_checkpoints':
    
            df = pd.read_csv(directory_path+file_name)
    
            df['tissue']=file_name.split('_EFs')[0]
    
            all_EFs_df=pd.concat([all_EFs_df, df])
            # Add your code here to do something with the file
    
    #check for convergence
    all_EFs_df = all_EFs_df[all_EFs_df['function_output_emp_bayes'].apply(lambda x: x.split('success: ')[1].split('\n')[0])=='True']
    
    #filter background
    all_EFs_background_SE = all_EFs_df[~all_EFs_df.passed_min_threshold_in_tissue]
    
    all_EFs_above_5_percent_PSI = all_EFs_df[all_EFs_df.passed_min_threshold_in_tissue]
    
    EF_5_bin = pd.cut(all_EFs_above_5_percent_PSI.EF_5, [0,0.1,0.3,0.5,0.7,0.9,1.0])
    
    all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(EF_5_bin=EF_5_bin)
    

    if type_ss=='alt_ss':
        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2
        
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='alt_splice_site')
        
        

    elif type_ss=='se':

            
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_1_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x:  x.split('_chr')[0]))
    ########
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_2_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[2]+':'+x.split(':')[3].split('_chr')[0]))
                                                                                                                                  
    
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_skipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1]))
    

        exclusion=all_EFs_above_5_percent_PSI[all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        # For 'exclusion' DataFrame
        intron_1_exclusion = exclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        exclusion = exclusion.assign(intron_1=intron_1_exclusion)
        
        intron_2_exclusion = exclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])


        exclusion = exclusion.assign(intron_2=intron_2_exclusion)
        
        # For 'inclusion' DataFrame
        inclusion=all_EFs_above_5_percent_PSI[~all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        intron_1_inclusion = inclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        inclusion = inclusion.assign(intron_1=intron_1_inclusion)


        
        intron_2_inclusion = inclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])
        inclusion = inclusion.assign(intron_2=intron_2_inclusion)
        
        # Combining 'inclusion' and 'exclusion' DataFrames
        all_EFs_above_5_percent_PSI = pd.concat([inclusion, exclusion])
        
                
        all_EFs_above_5_percent_PSI=pd.concat([inclusion, exclusion])

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='skipped_exon')
         
    top_EF_all_tissues = all_EFs_above_5_percent_PSI.sort_values(by='EF_5', ascending=False).drop_duplicates(subset='cluster_name_no_ID', keep='first')
    

    return all_EFs_above_5_percent_PSI, top_EF_all_tissues

        
   
        
                            
    

In [69]:
def filter_and_save_exons_new(type_ss):
    all_EFs_df=pd.DataFrame()
    directory_path = '/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/00_data/emp_bayes_output/'+type_ss+'_outputs/assigned_EFs/'
    files=os.listdir(directory_path)

    # Loop through each file in the directory
    for file_name in files:
        # Check if the entry is a file (not a directory)
        if file_name!= '.ipynb_checkpoints':
    
            df = pd.read_csv(directory_path+file_name)
    
            df['tissue']=file_name.split('_EFs')[0]
    
            all_EFs_df=pd.concat([all_EFs_df, df])
            # Add your code here to do something with the file
    
    #check for convergence
    all_EFs_df = all_EFs_df[all_EFs_df['function_output_emp_bayes'].apply(lambda x: x.split('success: ')[1].split('\n')[0])=='True']
    
    #filter background
    all_EFs_background_SE = all_EFs_df[~all_EFs_df.passed_min_threshold_in_tissue]
    
    all_EFs_above_5_percent_PSI = all_EFs_df[all_EFs_df.passed_min_threshold_in_tissue]
    
    EF_5_bin = pd.cut(all_EFs_above_5_percent_PSI.EF_5, [0,0.1,0.3,0.5,0.7,0.9,1.0])
    
    all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(EF_5_bin=EF_5_bin)
    

    if type_ss=='alt_ss':
        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2
        
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='alt_splice_site')
        
        

    elif type_ss=='se':

            
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_1_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x:  x.split('_chr')[0]))
    ########
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_2_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[2]+':'+x.split(':')[3].split('_chr')[0]))
                                                                                                                                  
    
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_skipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1]))
    

        exclusion=all_EFs_above_5_percent_PSI[all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        # For 'exclusion' DataFrame
        intron_1_exclusion = exclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        exclusion = exclusion.assign(intron_1=intron_1_exclusion)
        
        intron_2_exclusion = exclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])


        exclusion = exclusion.assign(intron_2=intron_2_exclusion)
        
        # For 'inclusion' DataFrame
        inclusion=all_EFs_above_5_percent_PSI[~all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        intron_1_inclusion = inclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        inclusion = inclusion.assign(intron_1=intron_1_inclusion)


        
        intron_2_inclusion = inclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])
        inclusion = inclusion.assign(intron_2=intron_2_inclusion)
        
        # Combining 'inclusion' and 'exclusion' DataFrames
        all_EFs_above_5_percent_PSI = pd.concat([inclusion, exclusion])
        
                
        all_EFs_above_5_percent_PSI=pd.concat([inclusion, exclusion])

        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='skipped_exon')

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)

        
         
    top_EF_all_tissues = all_EFs_above_5_percent_PSI.sort_values(by='EF_5', ascending=False).drop_duplicates(subset='cluster_name_no_ID', keep='first')
    
    all_EFs_above_5_percent_PSI.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_all_tissues_'+type_ss+'.csv.gz', 
                              compression = 'gzip',
                              index=True)
    

    
    top_EF_all_tissues.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/top_EF_all_tissues_'+type_ss+'.csv.gz', 
                              compression = 'gzip',
                              index=True)

    return all_EFs_above_5_percent_PSI, top_EF_all_tissues

        
   
        
                            
    
                              

In [70]:
[all_EFs_above_5_percent_PSI_se, top_EF_all_tissues_se]= filter_and_save_exons_new('se')




In [77]:
intron_1=all_EFs_above_5_percent_PSI_alt_ss.intron_1.apply(lambda x: x.split(':clu')[0])

intron_2=all_EFs_above_5_percent_PSI_alt_ss.intron_2.apply(lambda x: x.split(':clu')[0])
        

In [71]:
[all_EFs_above_5_percent_PSI_alt_ss, top_EF_all_tissues_alt_ss]= filter_and_save_exons_new('alt_ss')



sum stats
--

In [73]:
print('top EF SE# = '+str(len(top_EF_all_tissues_se)))

print('all EF SE# = '+str(len(all_EFs_above_5_percent_PSI_se)))

print('top EF altss# = '+str(len(top_EF_all_tissues_alt_ss)))

print('all EF altss# = '+str(len(all_EFs_above_5_percent_PSI_alt_ss)))

top EF SE# = 58579
all EF SE# = 278601
top EF altss# = 190746
all EF altss# = 793676


export this filtered set by EF, to filter for protein coding and maxent and size of exon in second (02) notebook
--

In [31]:
data_directory='/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/'

In [32]:
top_EF_all_tissues_se.to_csv(data_directory+'top_EF_all_tissues_se.csv.gz',
                                             compression='gzip',
                                             index=False)

top_EF_all_tissues_alt_ss.to_csv(data_directory+'top_EF_all_tissues_alt_ss.csv.gz',
                                             compression='gzip',
                                             index=False)


all_EFs_above_5_percent_PSI_se.to_csv(data_directory+'all_EF_all_tissues_se.csv.gz',
                                             compression='gzip',
                                             index=False)

all_EFs_above_5_percent_PSI_alt_ss.to_csv(data_directory+'all_EF_all_tissues_alt_ss.csv.gz',
                                             compression='gzip',
                                             index=False)